今天的目标是通过一个练习任务来探索scikit-learn的一些主要工具，任务是分析一个20个不同主题的文本集合（newsgroups posts）。
在这一节，我们将会学到：
*加载文件内容和类别
*提取适合机器学习的特征向量
*训练一个线性模型来执行分类
*使用一个grid search自动调参方法，来寻找对于特征提取成分和分类都不错的配置。

### Loading the 20 newsgroups dataset

数据集的名称是Twenty Newsgroups。这个数据集中大概有20000个新闻组文件，分为20个不同的组。一般用于文本分类和聚类的机器学习练习。

接下来我们会使用内置的dataset加载器来获取20newsgroups。或者，我们也可以手动下载数据集，然后利用sklearn.datasets.load_files函数来加载，注意要未压缩的存档文件夹。
为了加快执行速度，我们只使用4个类别，而不是全部20个类别。


In [1]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

我们现在可以加载匹配这几个分类的列表了：

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)


返回的数据集是一个scikit-learn的bunch：一个简单的具有域的holder对象，可以像python的dict对象一样访问，比如说target_names表示了请求的类别名：

In [3]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

文件本身加载在内存中，在data属性上访问。filenames也是可以访问的

In [4]:
 len(twenty_train.data)

2257

In [5]:
len(twenty_train.filenames)

2257

我们打印加载的第一个文件的一开始的行：

In [6]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [7]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


监督学习算法要求在测试集中为每个document提供一个label。在这个例子中，类别就是新闻组的名称，同时也是存储特定文件的文件夹的名称。
考虑到速度和空间的效率，scikit-learn将目标属性加载为一个整数数组，与target_names列表中的类别名称相对应。每个样本的类别整数id保存在target属性中：


In [8]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

还可能这样获取类别名称：

In [9]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


你可能注意到，样本已经被随机洗牌了：如果我们只选择一开始的一些样品去快速训练一个模型，并且没有在整个数据集上重复训练，这样随机打乱样本的顺序是有帮助的。

### Extracting features from text files
从文本中提取特征

为了执行文本文件的机器学习，我们需要将文本内容转化成数字特征向量。

#### Bag of words，BOW模型，也就是词袋模型。
最直观的方式就是用词袋模型：
*给训练集中任何一个文件中的每一个词，都指定一个固定的数字id，或者来说是建立一个从单词到数字的词典映射。
*对每一个文本#i，计算每个单词w在文本中出现的次数，并且保存在X[i，j]中，j表示在词典中单词w的序列号。

n_features表示在语料库中不同单词的数目，这个值一般都会大于100000.
如果n_samples==10000，存储X为一个numpy数组的话，需要RAM的4GB空间，对于先进的电脑来说不太能接受的。
幸运的是，X中的大部分值是0，因为一个给定的文档中，并不是所有单词都能被使用到。这样来讲，我们可以认为词袋是一个典型的高维稀疏数据集。我们可以保存很多内存，只存储特征向量的非零部分。
scipy.sparse矩阵就是这样的数据结构，scikit-learn内置了对这种结构的支持。

#### Tokenizing text with scikit-learn
用scikit-learn标记文本
停止词的文本预处理、标记和过滤包含在一个高级组件中，该组件能够构建一个特征字典并将文档转化为特征向量。


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

CountVectorizer支持N-gram的单词或者连续字符的计数。一旦安装好，向量机就建立了一个特征索引字典：

In [11]:
count_vect.vocabulary_.get(u'algorithm')

4690

一个单词的索引值与其在整个训练集中的频率有关。

#### From occurrences to frequencies

发生次数的统计是一个很好的开端，但是存在一个问题：越长的文档就会有更高的平均计数值，尽快他们会讨论相同的主题。
为了避免这些潜在的差异，将文档中每个单词的出现次数除以文档中的单词总数就可以了，得到的新特性成为术语频率，Term Frequency，简称tf。
在tf的基础上，另一个改进是对语料库中许多文档中出现的单词进行减重，因此比只出现在语料库较小部分中的单词的信息要少。
这个减重的操作称为tf-idf，Term Frequency times Inverse Document Frequency
tf和tf-idf都可以这么来计算：


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape


(2257, 35788)

在上面的范例代码中，我们首先使用fit()方法去拟合我们的预测器，然后使用transform()方法去转换我们的计数矩阵为一个tf-idf的表示。这两步可以整合来实现并加速处理，因为减少了多余处理过程。如下所示，使用fit_transform()方法。

In [13]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(2257, 35788)

### Training a classifier，训练一个分类器

现在我们已经有了自己的特征，我们就可以训练一个分类器来尝试预测一个文章的类别。让我们从一个pusu贝叶斯分类器开始，它为这个任务提供了一个很好的基线。scikit-learn包括这个分类器的几个变体：最适合单词计数的是多项式变体：


In [14]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)


为了预测新文档的结果，我们需要使用与以前几乎相同的特征提取链来提取特征。不同之处在于，我们使用transform而不是fit_transform，因为他们已经适合于训练集：

In [15]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))


'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


### Building a pipeline，创建一个pipeline

为了更方便的执行向量化——〉转换——〉分类的过程，scikit-learn提供了一个Pipelien类，就跟一个复合的分类器一样。


In [16]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
    ])


vect，tfidf和clf的名称是完全随意的，我们应当看到在前面网格搜索章节使用过pipeline。
我们现在可以使用一行命令来训练模型：


In [17]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### Evaluation of the performance on the test set，在测试集上评估表现

评估预测模型的准确度也是比较简单的：


In [18]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)


0.83488681757656458

我们得到了83.4%的准确率。让我们看看，如果使用一个线性SVM（被认为是最好的文本分类算法，尽快比朴素贝叶斯要慢一些），是否会得到更好的结果。只要简单的在pipeline中更换一下分类器即可：

In [19]:
>>> from sklearn.linear_model import SGDClassifier
>>> text_clf = Pipeline([('vect', CountVectorizer()),
... ('tfidf', TfidfTransformer()),
... ('clf', SGDClassifier(loss='hinge', penalty='l2',
... alpha=1e-3, random_state=42,
... max_iter=5, tol=None)),
... ])
>>> text_clf.fit(twenty_train.data, twenty_train.target)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [20]:
>>> predicted = text_clf.predict(docs_test)
>>> np.mean(predicted == twenty_test.target)


0.9127829560585885

scikit-learn提供更详细的结果分析工具：

In [21]:
>>> from sklearn import metrics
>>> print(metrics.classification_report(twenty_test.target, predicted,
... target_names=twenty_test.target_names))


                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [22]:
>>> metrics.confusion_matrix(twenty_test.target, predicted)

array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)

正如预期的那样，混乱矩阵显示，来自无神论和基督教新闻组的帖子常常彼此混淆，而计算机图形组的会较少。

### Parameter tuning using grid search
使用网格搜索算法对参数进行调节

我们已经在TfidfTransformer中遇到一些参数，类似于use_idf。分类器应该有很多的参数，类似于：MultinomialNB包括一个平滑参数alpha，而SGDClassifier在目标函数中有一个惩罚参数alpha和一个可配置损失和惩罚项。

与其调整chain中各个组件的参数，还不如在可能值得网格上对最佳参数进行彻底搜索。我们在单词或双体字上使用所有分类器，无论是否使用idf，对SVM的惩罚参数设置为0.01或者0.001：


In [23]:
>>> from sklearn.model_selection import GridSearchCV
>>> parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
... 'tfidf__use_idf': (True, False),
... 'clf__alpha': (1e-2, 1e-3),
... }


明显的，这样彻底的搜索代价是比较昂贵的。如果我们有多个CPU可供分配，我们可以告诉grid search，尝试这个八个参数的组合。如果我们设置n_jobs参数为-1，那么grid search会自动检测CPU：

In [24]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)

grid search实例表现的像一个普通的scikit-learn模型。让我们在一个较小的子集上运行这个search，加快计算：

In [25]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

得到一个分类器，我们可以用来预测：

In [26]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

对象的best_score_和best_params_属性存储了最佳的平均分数和对应的参数设置：

In [27]:
gs_clf.best_score_

0.90000000000000002

In [29]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))


clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)


grid search的更详细的总结可以通过.cv_results_来获取。